In [1]:
import streamlit as st
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.agents import AgentType, create_sql_agent
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_core.prompts import PromptTemplate
from IPython.display import Markdown as md

In [2]:
# Read the API key
f = open("keys/.gemini_api_key.txt")
key = f.read()

In [3]:
#  Setup chat model
chat_model = ChatGoogleGenerativeAI(google_api_key=key, model="gemini-1.5-pro-latest")

In [4]:
# connect to our database
db = SQLDatabase.from_uri("sqlite:///laptops.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT DISTINCT brand FROM laptops;")

sqlite
['laptops']


"[('HP',), ('Lenovo',), ('ASUS',), ('RedmiBook',), ('acer',), ('Infinix',), ('APPLE',), ('DELL',), ('MSI',), ('SAMSUNG',), ('realme',), ('ALIENWARE',), ('GIGABYTE',), ('Avita',), ('Nokia',), ('Vaio',), ('Mi',), ('LG',), ('Ultimus',)]"

In [5]:
db.run("SELECT DISTINCT storage FROM laptops;")

"[('SSD',), ('HDD',), ('EMMC',)]"

In [6]:
db.run("SELECT brand, max(price) FROM laptops;")

"[('DELL', 1174131)]"

In [7]:
db.run("SELECT AVG(price) FROM laptops WHERE brand = 'Lenovo' AND processor = 'Core i5';")

'[(68510.26666666666,)]'

In [8]:
prompt = PromptTemplate.from_template(
    """You are a helpful AI assistant expert in querying SQL Database,
        The Database has the name laptops
        and has the following columns - brand, description, model, price, RAM(GB), storage,
        capacity(GB), processor, warranty(years) and display(inch). The price is in Naira.
        DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.
        if a question does not seem related to the database, the response should be,
        "I may not be able to provide information about this topic"
        Given the following user question, generate corresponding SQL query, and SQL result, 
        and answer the user question. 
        Question: {question}
        Answer: 
        """
)

In [9]:
sql_toolkit = SQLDatabaseToolkit(db=db, llm=chat_model)
sql_toolkit.get_tools()

sqldb_agent = create_sql_agent(
    llm=chat_model,
    toolkit=sql_toolkit,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [10]:
response = sqldb_agent.run(prompt.format(
        question="what are the available brands?"
  ))
response

c:\Users\USER\Documents\Data Science and Analytics\InnomaticsResearchLabs\langchain_rag\.langchain_env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SQL Agent Executor chain...


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


In [ ]:
md(response)

The available brands are: HP, Lenovo, ASUS, RedmiBook, acer, Infinix, APPLE, DELL, MSI, SAMSUNG, realme, ALIENWARE, GIGABYTE, Avita, Nokia, Vaio, Mi, LG, Ultimus

In [ ]:
response = sqldb_agent.run(prompt.format(
        question="What is the average price of hp core i5 processor laptop?"
  ))
response



> Entering new SQL Agent Executor chain...
Thought: I need to query the database to find the average price of HP laptops with Core i5 processors. 
Action: sql_db_query_checker
Action Input: SELECT AVG(price) FROM laptops WHERE brand = 'HP' AND processor LIKE '%Core i5%' ```sql
SELECT AVG(price) FROM laptops WHERE brand = 'HP' AND processor LIKE '%Core i5%'
```

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 16.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 32.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


Action: sql_db_query
Action Input: SELECT AVG(price) FROM laptops WHERE brand = 'HP' AND processor LIKE '%Core i5%'[(73819.0,)]Final Answer: The average price of an HP laptop with a Core i5 processor is ₦73819.0. 


> Finished chain.


'The average price of an HP laptop with a Core i5 processor is ₦73819.0.'

In [ ]:
md(response)

The average price of an HP laptop with a Core i5 processor is ₦73819.0.